In [ ]:
!pip install contractions transformers torch sentencepiece accelerate>=0.20.1 datasets tqdm evaluate huggingface_hub nltk

In [ ]:
import nltk
import urllib
import bs4 as bs
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
from gensim.models import Word2Vec
import pandas as pd # Pandas is for working with tables
from imblearn.over_sampling import RandomOverSampler #Over sampling
import plotly.graph_objects as go #Graph
from sklearn.preprocessing import LabelEncoder #Converts category to numerical
from imblearn.under_sampling import RandomUnderSampler #Under sampling
import re
import contractions
from google.colab import drive
import torch
from transformers import AutoTokenizer
import numpy as np
from nltk.tokenize import word_tokenize
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from transformers import AutoTokenizer,AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoConfig
from datasets import load_metric
from tqdm import tqdm
from torch.utils.data import Dataset
import numpy as np
import torch.nn as nn
from huggingface_hub import notebook_login, HfApi
import os
import requests

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Load dataset and analyse it

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -O dataset.csv "https://drive.google.com/uc?export=download&id=1jNxz6goLsrcCEQYzcie3tuKLDlois_Uj&confirm=NEW_FILE_CONFIRM_CODE"

--2023-07-13 21:38:54--  https://drive.google.com/uc?export=download&id=1jNxz6goLsrcCEQYzcie3tuKLDlois_Uj&confirm=NEW_FILE_CONFIRM_CODE
Resolving drive.google.com (drive.google.com)... 142.250.136.100, 142.250.136.139, 142.250.136.138, ...
Connecting to drive.google.com (drive.google.com)|142.250.136.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-c8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/98tuj0387hdni7ivu8jmpjec714kvgcn/1689284325000/12434615743218103933/*/1jNxz6goLsrcCEQYzcie3tuKLDlois_Uj?e=download&uuid=55cb96fb-e388-4b56-aea6-ac9daef9c589 [following]
--2023-07-13 21:38:56--  https://doc-08-c8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/98tuj0387hdni7ivu8jmpjec714kvgcn/1689284325000/12434615743218103933/*/1jNxz6goLsrcCEQYzcie3tuKLDlois_Uj?e=download&uuid=55cb96fb-e388-4b56-aea6-ac9daef9c589
Resolving doc-08-c8-docs.googleusercontent.com (doc-08-c8-docs.googleusercon

In [ ]:
dataframe = pd.read_csv('/content/dataset.csv')
dataframe[0:10]

,content,sentiment
0,i know i was listenin to bad habit earlier an...,2
1,layin n bed with a headache ughhhhwaitin on y...,4
2,funeral ceremonygloomy friday,4
3,wants to hang out with friends soon,3
4,we want to trade with someone who has houston ...,2
5,re-pinging : why did not you go to prom becaus...,6
6,i should be sleep but i am not thinking about ...,4
7,hmmm is down,6
8,charlene my love i miss you,4
9,i am sorry at least it is friday,4


In [ ]:
dataframe['sentiment'].value_counts()

2    9644
6    8459
3    7744
4    5165
1    3842
0    2774
5    2759
Name: sentiment, dtype: int64

### Check who needs oversampling and who needs undersampling

In [ ]:
X_values = dataframe[['content']]
y_values = dataframe['sentiment']
class_counts = y_values.value_counts()
categories = list(class_counts.index)
average_count = class_counts.mean()
average_count = int(average_count)
average_count

5769

In [ ]:
oversampler = RandomOverSampler(random_state=42, sampling_strategy= { element:average_count for element in [categories[index] for index, i in enumerate(class_counts) if i < average_count]})
x_over_sample, y_over_sample = oversampler.fit_resample(X_values, y_values)

In [ ]:
undersampler = RandomUnderSampler(random_state=42, sampling_strategy={ element:average_count for element in [categories[index] for index, i in enumerate(class_counts) if i >= average_count]})
X_balanced, y_balanced  = undersampler.fit_resample(x_over_sample, y_over_sample)

## PreProcessing of tweets

In [ ]:
def remove_stopwords(sentences):
    stop_words = set(stopwords.words('english'))
    filtered_sentences = []

    for sentence in sentences:
      words = word_tokenize(sentence)
      temp = [word for word in words if word not in stop_words]
      filtered_sent = ' '.join(temp)
      filtered_sentences.append(filtered_sent)

    return filtered_sentences

def remove_punctuation(sentences):
    punctuation = string.punctuation
    filtered_sentences = []

    for sentence in sentences:
      temp = [char for char in sentence if char not in punctuation]
      new_sent = ''.join(temp)
      filtered_sentences.append(new_sent)
    return filtered_sentences

def stem_sentences(sentences):
    stemmer = PorterStemmer()
    stemmed_sentences = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        temp = [stemmer.stem(word) for word in words]
        stemmed_sent = ' '.join(temp)
        stemmed_sentences.append(stemmed_sent)
    return stemmed_sentences

def lem_sentences(sentences):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        temp = [lemmatizer.lemmatize(word) for word in words]
        stemmed_sent = ' '.join(temp)
        lemmatized_sentences.append(stemmed_sent)

    return lemmatized_sentences

def preprocess_sentence(sentences):
    # Remove punctuation
    sentences = remove_punctuation(sentences)
    sentences = remove_stopwords(sentences)
    sentences = stem_sentences(sentences)
    sentences = lem_sentences(sentences)
    # WRITE YOUR CODE HERE #
    return sentences

def clean_content(text):

  # remove twitter handles
  clean_text = re.sub(r'@\w+\s?', "", str(text))

  # convert to lowercase
  clean_text = clean_text.lower()

  # remove links http:// or https://
  clean_text = re.sub(r'https?:\/\/\S+', '', clean_text)

  # remove links beginning with www. and ending with .com
  clean_text = re.sub(r'www\.[a-z]?\.?(com)+|[a-z]+\.(com)', '', clean_text)

  # remove html reference characters
  clean_text = re.sub(r'&[a-z]+;', '', clean_text)

  # remove non-letter characters besides spaces "/", ";" "[", "]" "=", "#" Regex syntax
  clean_text = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', clean_text)

  clean_text = contractions.fix(clean_text)
  sentences = sent_tokenize(clean_text)
  filtered_sentences = preprocess_sentence(sentences)
  clean_text = '.'.join(filtered_sentences)
  return clean_text

In [ ]:
max_len = X_balanced.shape[0]
for i in range(max_len):
  X_balanced.iloc[i,0] = clean_content(X_balanced.iloc[i,0])

In [ ]:
X_balanced[0:10]

,content
0,annoy start type comput middl night
1,dammit hulu desktop total screw abil talk part...
2,cayogi want come bz summer sure anymor teacher...
3,win sigh rakeem
4,damn suck
5,websit gave viru open window kept pop
6,waraku tasteless expens portion littl waraku
7,youstinkatrespondingtotext
8,hate finish work still hour left work time
9,umm yeah probabl pretti good note self eeeeeew...


## Tokenize data

In [ ]:
max_string_len = 0
for row in range(max_len):
  row_text = X_balanced.iloc[row,0]
  words = word_tokenize(str(row_text))
  if len(words) > max_string_len:
    max_string_len = len(words)
max_string_len

42

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
input_ids = []
attention_masks = []
y_label = []
for i in range(0,max_len):
  text = X_balanced.iloc[i,0]
  label = y_balanced.iloc[i]
  tokens = tokenizer(str(text), truncation=True, padding="max_length", max_length=max_string_len+5, return_tensors="pt")
  input_ids.append(tokens['input_ids'][0])
  attention_masks.append(tokens['attention_mask'][0])
  y_label.append(label)

## Create dataset

In [ ]:
class TweetDataset(Dataset):
    def __init__(self, input_ids, attention_masks, y_labels):
        super().__init__()
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.y_labels = y_labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        ans = {
              'input_ids': self.input_ids[index],
              'attention_mask': self.attention_masks[index],
              'labels': torch.tensor([self.y_labels[index]], dtype=torch.long)
          }
        return ans

In [ ]:
tweet_dataset = TweetDataset(input_ids, attention_masks, y_label)

## Train model

In [ ]:
models = ["nlptown/bert-base-multilingual-uncased-sentiment",
          "distilbert-base-uncased",
          "Seethal/sentiment_analysis_generic_dataset",
          "microsoft/deberta-large-mnli"]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(models[2])
id2label = {0: 'anger', 1: 'love', 2: 'neutral', 3: 'positive', 4: 'sadness', 5: 'surprise', 6: 'worry'}
label2id = {'anger':0, 'love':1,'neutral':2, 'positive':3, 'sadness':4, 'surprise':5, 'worry': 6}
num_labels = 7

In [ ]:
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, num_labels)
)
model.config.num_labels = num_labels
model.config.id2label = id2label
model.config.label2id = label2id
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

Just checking Pretrained model

In [ ]:
demo_input_id = torch.tensor([(tweet_dataset[0]['input_ids']).tolist()])
demo_attention_mask = torch.tensor([(tweet_dataset[0]['attention_mask']).tolist()])
expected_output = tweet_dataset[0]['labels']
output = model(input_ids=demo_input_id, attention_mask=demo_attention_mask).logits
loss_function = nn.CrossEntropyLoss()
loss = loss_function(output, expected_output)
print(f"Output is {output} and loss is {loss}")

Output is tensor([[ 0.0814,  0.0566, -0.2332, -0.0525,  0.1196,  0.4231, -0.3067]],
       grad_fn=<AddmmBackward0>) and loss is 1.9028339385986328


In [ ]:
train_size = int(0.8 * len(tweet_dataset))
train_data, val_data = random_split(tweet_dataset, [train_size, len(tweet_dataset) - train_size])
learning_rate = 1e-5
num_epochs = 15
batch_size = 100
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [ ]:
model.to('cuda')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
def validation_accuracy():
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch in val_loader:
            batch_input_ids = batch["input_ids"].to('cuda')
            batch_attention_mask = batch["attention_mask"].to('cuda')
            batch_labels = batch["labels"].to('cuda')

            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            predicted_labels = torch.argmax(logits, dim=1)

            # Count the correct predictions
            correct_predictions += (predicted_labels == batch_labels.squeeze()).sum().item()
            total_predictions += batch_labels.size(0)

    accuracy = correct_predictions / total_predictions
    return accuracy

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        batch_input_ids = batch["input_ids"].to('cuda')
        batch_attention_mask = batch["attention_mask"].to('cuda')
        batch_labels = batch["labels"].to('cuda')
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask).logits
        loss = loss_function(outputs, batch_labels.squeeze())
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    val_loss = validation_accuracy()
    print(f"Epoch {epoch+1}: Average training Loss = {avg_loss} Average validation accuracy = {val_loss}")
    model.save_pretrained("/content/model")

Epoch 1: Average training Loss = 1.4330190846949449 Average validation accuracy = 0.4819858858487062
Epoch 2: Average training Loss = 1.3550993998845418 Average validation accuracy = 0.49659527052123315
Epoch 3: Average training Loss = 1.2769109065885897 Average validation accuracy = 0.5136808220874087
Epoch 4: Average training Loss = 1.1934312530505804 Average validation accuracy = 0.5254426148322396
Epoch 5: Average training Loss = 1.10803404652778 Average validation accuracy = 0.5333663488919154
Epoch 6: Average training Loss = 1.0226233772657536 Average validation accuracy = 0.5466138417729355
Epoch 7: Average training Loss = 0.9417320777328662 Average validation accuracy = 0.5541661507985638
Epoch 8: Average training Loss = 0.8694163834975089 Average validation accuracy = 0.555651850934753
Epoch 9: Average training Loss = 0.7958796127342883 Average validation accuracy = 0.5587470595518138
Epoch 10: Average training Loss = 0.7332736524166884 Average validation accuracy = 0.56258511

In [ ]:
model.to('cpu')
from transformers import pipeline
text = "I hate you, you are disgusting"
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(text, top_k=3)

[{'label': 'anger', 'score': 0.7275619506835938},
 {'label': 'sadness', 'score': 0.23586194217205048},
 {'label': 'worry', 'score': 0.018645161762833595}]

In [ ]:
tokenizer.save_pretrained("/content/model")

('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.txt',
 '/content/model/added_tokens.json',
 '/content/model/tokenizer.json')

## Upload to hugging face

In [ ]:
api = HfApi()
notebook_login()

In [ ]:
#To-Do: use your repo_id
api.create_repo(repo_id="tweet_sentiments_40k_new")

RepoUrl('https://huggingface.co/pzalavad/tweet_sentiments_40k_new', endpoint='https://huggingface.co', repo_type='model', repo_id='pzalavad/tweet_sentiments_40k_new')

In [ ]:
#To-Do: use your repo_id
repo_id="pzalavad/tweet_sentiments_40k_new"
folder_path = '/content/model'
files = os.listdir(folder_path)
for file_name in files:
  api.upload_file(
      path_or_fileobj=os.path.join(folder_path, file_name),
      path_in_repo=file_name,
      repo_id=repo_id,
      repo_type="model",
  )

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

## Test model

In [ ]:
repo_id="pzalavad/tweet_sentiments_40k_new"

In [ ]:
def query(API_URL,
          payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

In [ ]:
#To-Do: Add your own token
token = ""
test_tweets = {"I can't believe how disrespectful some people can be! It's infuriating! #Angry #Frustrated #Outrage": 'anger',
          "Just had the worst customer service experience ever! Feeling furious right now. #AngryCustomer #UnbelievableService": 'anger',
          "I am so lucky to have found my soulmate. Every day with you feels like a dream come true. #Love #Grateful #Blessed": 'love',
          "Love is the most beautiful feeling in the world. It makes everything brighter and better. Spread love everywhere you go. #SpreadLove #LoveIsAllYouNeed": 'love',
          "Just finished reading an interesting book. It was a thought-provoking journey. 📚 #BookLover #Thoughtful #Recommendation": 'neutral',
          "The weather today is neither too hot nor too cold. Just perfect for a walk in the park. 🌤️ #WeatherUpdate #EnjoyingNature": 'neutral',
          "I woke up feeling incredibly motivated and ready to conquer the day! Let's make it a productive one, folks! 💪 #Motivated #PositiveVibes #GoGetIt": 'positive',
          "Received some great news today! My hard work finally paid off. Feeling on top of the world right now. 🌟 #AchievementUnlocked #HappyDance": 'positive',
          "Feeling heartbroken today. It's hard to let go of something you held so dear. 💔 #Heartbroken #Emotional #Sadness": 'sadness',
          "Sometimes life throws curveballs, and it's okay to feel sad. Just remember to take it one step at a time and lean on your loved ones for support. 🌸 #SadButStrong #SupportSystem": 'sadness',
          "OMG! I just got the most amazing surprise from my friends! They truly know how to make me feel special. 🎉 #Surprised #Blessed #FriendsForever": 'surprise',
          "Didn't expect to find a hidden gem of a restaurant in this small town. The food blew my mind! Highly recommend checking it out. 🍽️ #HiddenGem #FoodieDiscoveries": 'surprise',
          "Feeling worried about the upcoming exams. So much to study, so little time. 😰 #StressedOut #ExamSeason #Nervous": 'worry',
          "Just heard about the recent spike in COVID cases. It's concerning, and we need to take all necessary precautions. Stay safe, everyone! 🙏 #COVID19 #StaySafe #Precautions": 'worry'
}

preprocessed_tweets = [ clean_content(element) for element in test_tweets.keys()]
labels = [ element for element in test_tweets.values()]
headers = {"Authorization": "Bearer "+token}

In [ ]:
preprocessed_tweets

['believ disrespect peopl infuri angri frustrat outrag',
 'worst custom servic experi ever feel furiou right angrycustom unbelievableservic',
 'lucki found soulmat everi day feel like dream come true love grate bless',
 'love beauti feel world make everyth brighter better spread love everywher go spreadlov loveisallyoune',
 'finish read interest book thoughtprovok journey booklov thought recommend',
 'weather today neither hot cold perfect walk park weatherupd enjoyingnatur',
 'woke feel incred motiv readi conquer day let u make product one folk motiv positivevib gogetit',
 'receiv great news today hard work final paid feel top world right achievementunlock happyd',
 'feel heartbroken today hard let go someth held dear heartbroken emot sad',
 'sometim life throw curvebal okay feel sad rememb take one step time lean love one support sadbutstrong supportsystem',
 'omg got amaz surpris friend truli know make feel special surpris bless friendsforev',
 'expect find hidden gem restaur small 

In [ ]:
API_URL = "https://api-inference.huggingface.co/models/"+str(repo_id)
final_score = 0
for index,tweet in enumerate(preprocessed_tweets):
  output = query(API_URL,
                {"inputs": tweet}
                )
  print(output)
  pred_labels = [item['label'] for item in output[0][:3]]
  print(pred_labels)
  if labels[index] in pred_labels:
    final_score += 1
print(f"Final score of {repo_id} is {final_score}")

[[{'label': 'worry', 'score': 0.2007777988910675}, {'label': 'surprise', 'score': 0.18376997113227844}, {'label': 'sadness', 'score': 0.1541154384613037}, {'label': 'love', 'score': 0.12621766328811646}, {'label': 'neutral', 'score': 0.12604449689388275}, {'label': 'positive', 'score': 0.10571947693824768}, {'label': 'anger', 'score': 0.10335516184568405}]]
['worry', 'surprise', 'sadness']
[[{'label': 'anger', 'score': 0.19951382279396057}, {'label': 'surprise', 'score': 0.1878957748413086}, {'label': 'love', 'score': 0.1598195880651474}, {'label': 'worry', 'score': 0.15493012964725494}, {'label': 'sadness', 'score': 0.10359756648540497}, {'label': 'positive', 'score': 0.10280165076255798}, {'label': 'neutral', 'score': 0.09144143015146255}]]
['anger', 'surprise', 'love']
[[{'label': 'love', 'score': 0.23799538612365723}, {'label': 'positive', 'score': 0.17078305780887604}, {'label': 'worry', 'score': 0.16789740324020386}, {'label': 'anger', 'score': 0.12203629314899445}, {'label': 'sa